# Kategorien für Gastronomie-Feedbacks ableiten

[TeLLers](https://tellers.co.at/) ist eine mobile Web-Applikation um Feedback für Gastronomiebetriebe zu sammeln. Im Unterschied zu zahlreichen Web-Seiten mit einem ähnlichen Ziel sind diese Feedbacks nicht im Internet öffentlich zugänglich. Stattdessen kann sie nur der Gastronom selbst einsehen und damit gezielt seine Produkte und Dienstleistungen den Kundenbedürfnissen anpassen, ohne sich in öffentliche Diskussionen und potentielle "Shot storms" zu verstricken.

Dieses Notebook zeigt, wie bestehende Feedback-Einträge analysiert werden können, um darauf Aufsetzend ein Kategoriesystem zu entwickeln.

## Überblick

Die Feedbackdaten liegen in einer SQLite-Tabelle `feedback` in der Spalte `text`. Unter verwendung von Python 3.5+ landen die Daten in einer Liste, werden mit Funktionen aus der der Bibliothek [nltk](https://www.nltk.org/) in Sätze und Wörter zerlegt und dann mit [gensim](https://radimrehurek.com/gensim/) nach darin enthaltenen Themen untersucht. Die vorliegende Analyse basiert auf dem Gensim-Tutorial zu "[Topics and Transformations](https://radimrehurek.com/gensim/tut2.html)".

## Vorbereitung

Um Log-Ausgaben übersichtlich darzustellen sind folgende Einstellungen sinnvoll:

In [1]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)

Folgende Funktion wird später verwendet, um eine Liste von Listen von Wörtern um leere Wortlisten zu bereinigen:

In [2]:
from typing import List
def clean_texts(texts: List[List[str]]) -> List[List[str]]:
    text_index = 0
    while len(texts) > text_index:
        if len(texts[text_index]) == 0:
            del texts[text_index]
        else:
            text_index += 1
    logging.info('%d texts remaining', len(texts))

Import gensim:

In [3]:
from gensim import corpora, models

INFO : 'pattern' package not found; tag filters are not available for English


## Feedback-Texte einlesen

Da die Daten als SQLite-Datenbank aufliegen (Tabelle bzw. Feld `feedback.text`) lassen sie sich wie folgt in eine Python-Liste `feedback_texts` übertragen:

In [4]:
import sqlite3
from contextlib import closing

with sqlite3.connect('/tmp/tellers.db') as tellers_db:
    with closing(tellers_db.cursor()) as tellers_cursor:
        tellers_cursor.execute("""
            select 
                text 
            from 
                feedback 
            where 1 = 1
                and source_id = 1  -- consider only TeLLers feedback
                and length(text) >= 20
                and lower(text) not like '%test%'
            order by
                feedback_time desc  -- newer feedback first
        """)
        feedback_texts = [row[0] for row in tellers_cursor.fetchall()]
logging.info('found %d feedbacks', len(feedback_texts))

INFO : found 562 feedbacks


Das Ergebnis ist eine Liste wie:

In [5]:
feedback_texts[:3]

['Freundliche Bedingung.\\nGute Qualität der Speisen.',
 'Das Interior modernisieren',
 'Eine vegane Torte oder ein Veganer Kuchen']

## Texte in Sätze und Wörter zerlegen

Im ersten Schritte sind die potentiell langen Feedbacks in Sätze zu zerlegen. Dazu ist die Funktion `nltk.sent_tokenize()` hilfreich, da sie auch mit fortgeschrittenen Konzepten wie der indirekten Sprache umgehen kann:

In [6]:
import nltk
sentences = []
for feedback_text in feedback_texts:
    feedback_sentences = nltk.sent_tokenize(feedback_text)
    sentences.extend(feedback_sentences)
logging.info('found %d feedback sentences', len(sentences))

INFO : found 904 feedback sentences


Dokumente in Text-Token umwandeln:

In [7]:
texts = [nltk.word_tokenize(sentence) for sentence in sentences]
logging.info('found %d texts with words', len(texts))

INFO : found 904 texts with words


## Nur mit Großbuchstaben beginnende Wörte behalten (obsolet)

Um in vereinfachter Form nur Hauptwörter für die Analyse zu behalten, ist eine pragmatische Abkürzung, die Texte auf Wörter zu reduzieren, die mit Großbuchstaben beginnen. Dies ist allerdings nicht ganz korrekt, da auch der Satzanfang oder Substantivierungen mit Großbuchstaben beginnen.

> Hinweis: da dieser Filter die Qualität der Analyse eher nagativ beeinflusst, wurde er wieder deaktiviert. Da er für andere Anwendungen sinnvoll sein kann, ist der deaktivierte Code im folgenden dennoch angeführt.

In [8]:
if False:
    texts = [[word for word in text if word[0].upper() == word[0]]
             for text in texts]
    clean_texts(texts)

## Nur Text-Wörter behalten und in Kleinschrift umwandeln

Um Satzzeichen und Zahlen zu entfernen, bietet es such an, nur jene Wörter zu behalten, die mit einem Buchstaben beginnen. Hilfreich dazu ist die Funktionen `str.isalpha()`, die übrigens auch Umlaute erkennt. Im Zuges dieses Schritts erfolgt auch eine Umwandlung der Wörter in Kleinschrift, da die darauffolgenden Filter damit einfacher umzusetzen sind.

In [9]:
texts = [[word.lower() for word in text if word[0].isalpha()]
         for text in texts]
clean_texts(texts)

INFO : 868 texts remaining


## Stoppwörter entfernen

Stoppwörter sind Wörter, welche für die Analyse keine Relevanz haben, zum Beispiel bestimmte Artikel wie "der" oder Bindewörter wie "oder". Das Projekt [stopwords-iso](https://github.com/stopwords-iso/stopwords-iso) sammelt Stoppwörter für verschiedene Sprachen. Mit Hilfe des [requests](http://docs.python-requests.org/)-Modules ist es kompakt mögliche, die aktuellste Version der deutschen Stoppwörter eine eine Python-Menge (`set`) zu übertragen:

In [10]:
import requests
query = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-de/master/stopwords-de.txt')
query.raise_for_status()
stopwords = set(query.text.split('\n'))
print(sorted(stopwords)[:10])  # Print inital 10 stopwords.

['a', 'ab', 'aber', 'ach', 'acht', 'achte', 'achten', 'achter', 'achtes', 'ag']


Damit können die Stoppwörter aus den Feedback-Texten entfernt werden:

In [11]:
texts = [[word for word in text if not word in stopwords]
         for text in texts]
clean_texts(texts)

INFO : 849 texts remaining


## Nur einmal vorkommende Wörter entfernen

Wörter, die in allen Feedbacks nur einmal vorkommen, sind so speziell, dass sie auf das Ergebnis keinen Einfluss haben. Um die Performanze zu verbessern, kann man diese noch vor der Analyse entfernen. Die Standard-Module `collections` und `iterable` enthalten dazu hilfreiche Funktionen, um eine Liste von Liste zu "verflachen" und die Häufigkeit der darin enthaltenene Texte zu ermitteln.

In [12]:
from collections import Counter
from itertools import chain
word_counter = Counter(chain.from_iterable(texts))
once_words = set(word for word, count in word_counter.items() if count == 1)
texts = [[word for word in text if word not in once_words]
         for text in texts]
clean_texts(texts)

INFO : 751 texts remaining


## Zwischenergebnis

Als Ergenis der bisherigen Umwandlungen eralten wir eine Liste von Wortlisten, die Beispielhaft wie folgt aussieht:

In [13]:
texts[:10]

[['freundliche', 'qualität', 'speisen'],
 ['modernisieren'],
 ['vegane', 'torte', 'veganer'],
 ['super', 'restaurant'],
 ['gedicht'],
 ['essen', 'super', 'schnell'],
 ['gerne'],
 ['bestens'],
 ['eventuell', 'pizza'],
 ['danke', 'gerne', 'trotz', 'freundliche', 'bedienung']]

## Analyse mit gensim

Um die nun als Wortlisten vorliegenden Texte zu mit gensim analysieren, sind mehrere Schritte erforderlich.

Zuerste wird daraus ein Wörterbuch für den Corpus erstellt:

In [14]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/gastro_feedback.dict')

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(474 unique tokens: ['freundliche', 'qualität', 'speisen', 'modernisieren', 'torte']...) from 751 documents (total 1930 corpus positions)
INFO : saving Dictionary object under /tmp/gastro_feedback.dict, separately None
INFO : saved /tmp/gastro_feedback.dict


Damit lässt sich der tatsächliche Corpus aufbauen:

In [15]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)

INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
INFO : saving sparse matrix to /tmp/deerwester.mm
INFO : PROGRESS: saving document #0
INFO : saved 751x474 matrix, density=0.531% (1892/355974)
INFO : saving MmCorpus index to /tmp/deerwester.mm.index


Aus dem Corpus lässt sich die "[term frequency–inverse document frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (TFIDF) berechnen. Dabei handelt es sich um eine Kennzahl, die jedem Wort eine Wichtigkeit zuordnet. Basis dafür ist, wie oft das Wort in einem Feedback-Text und wie oft es im gesamten Corpus auftritt.

In [16]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : calculating IDF weights for 751 documents and 473 features (1892 matrix non-zeros)


Der so transformierte Corpus kann nun als Basis für einen "[latent semantic index](https://en.wikipedia.org/wiki/Latent_semantic_analysis)" (LSI) dienen. 

In [17]:
TOPIC_COUNT = 6
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=TOPIC_COUNT)
corpus_lsi = lsi[corpus_tfidf]

INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (474, 106) action matrix
INFO : orthonormalizing (474, 106) action matrix
INFO : 2nd phase: running dense svd on (106, 751) matrix
INFO : computing the final decomposition
INFO : keeping 6 factors (discarding 83.464% of energy spectrum)
INFO : processed documents up to #751
INFO : topic #0(4.002): -0.789*"super" + -0.459*"essen" + -0.214*"kellner" + -0.174*"personal" + -0.102*"musik" + -0.093*"bedienung" + -0.074*"top" + -0.073*"echt" + -0.064*"qualität" + -0.063*"lecker"
INFO : topic #1(3.669): -0.836*"essen" + 0.439*"super" + 0.210*"kellner" + -0.087*"top" + -0.076*"ambiente" + -0.074*"schmeckt" + 0.072*"musik" + 0.055*"personal" + -0.051*"tolle" + -0.050*"teuer"
INFO : topic #2(3.252): 0.904*"kellner" + -0.268*"super" + -0.168*"personal" + 0.155*"blonde" + 0.081*"essen" + 0

Das Ergebnis ist eine Liste von Themen (ohne Namen, nur mit Index 0, 1, 2, ...) sowie der wichtigsten darin enthaltenen Wörter und Wortkombinationen:

In [18]:
lsi.print_topics(TOPIC_COUNT)

INFO : topic #0(4.002): -0.789*"super" + -0.459*"essen" + -0.214*"kellner" + -0.174*"personal" + -0.102*"musik" + -0.093*"bedienung" + -0.074*"top" + -0.073*"echt" + -0.064*"qualität" + -0.063*"lecker"
INFO : topic #1(3.669): -0.836*"essen" + 0.439*"super" + 0.210*"kellner" + -0.087*"top" + -0.076*"ambiente" + -0.074*"schmeckt" + 0.072*"musik" + 0.055*"personal" + -0.051*"tolle" + -0.050*"teuer"
INFO : topic #2(3.252): 0.904*"kellner" + -0.268*"super" + -0.168*"personal" + 0.155*"blonde" + 0.081*"essen" + 0.061*"bedienung" + -0.060*"lokal" + 0.054*"netter" + 0.053*"könnten" + 0.048*"is"
INFO : topic #3(3.052): 0.732*"personal" + 0.512*"lokal" + -0.239*"super" + 0.173*"musik" + 0.114*"getränke" + 0.107*"freundliches" + 0.086*"kellner" + 0.079*"angebot" + 0.072*"tisch" + 0.062*"bedienung"
INFO : topic #4(2.924): -0.802*"musik" + -0.241*"top" + -0.217*"laut" + -0.199*"gerne" + -0.190*"perfekt" + 0.172*"personal" + 0.147*"lokal" + 0.105*"kellner" + -0.102*"unterhalten" + -0.082*"leisere"
I

[(0,
  '-0.789*"super" + -0.459*"essen" + -0.214*"kellner" + -0.174*"personal" + -0.102*"musik" + -0.093*"bedienung" + -0.074*"top" + -0.073*"echt" + -0.064*"qualität" + -0.063*"lecker"'),
 (1,
  '-0.836*"essen" + 0.439*"super" + 0.210*"kellner" + -0.087*"top" + -0.076*"ambiente" + -0.074*"schmeckt" + 0.072*"musik" + 0.055*"personal" + -0.051*"tolle" + -0.050*"teuer"'),
 (2,
  '0.904*"kellner" + -0.268*"super" + -0.168*"personal" + 0.155*"blonde" + 0.081*"essen" + 0.061*"bedienung" + -0.060*"lokal" + 0.054*"netter" + 0.053*"könnten" + 0.048*"is"'),
 (3,
  '0.732*"personal" + 0.512*"lokal" + -0.239*"super" + 0.173*"musik" + 0.114*"getränke" + 0.107*"freundliches" + 0.086*"kellner" + 0.079*"angebot" + 0.072*"tisch" + 0.062*"bedienung"'),
 (4,
  '-0.802*"musik" + -0.241*"top" + -0.217*"laut" + -0.199*"gerne" + -0.190*"perfekt" + 0.172*"personal" + 0.147*"lokal" + 0.105*"kellner" + -0.102*"unterhalten" + -0.082*"leisere"'),
 (5,
  '-0.617*"top" + 0.361*"musik" + -0.303*"speisen" + -0.259*"

## Interpretation

Die Daten ergeben leider keine Eindeutigen Trends, da wiederholt ähnliche Begriffe die Basis für die unterschiedlichen Kategorien sind. Mit der Konstante `TOPIC_COUNT` lässt sich experimentieren. Der Wert 6 scheint hier am einfachsten interpretierbaren Ergebnisse zu liefern.

Eine mögliche Interpretation ist:

* Essen: topic 1
* Service: topic 2, topic 3
* Musik: topic 4

Klar ist, dass eine solche Analyse nur eine Basis für eine Kategoriesystem sein kann. Insbesondere aufgrund der vergleichsweise geringen Datenanzahl ist mit einer gewissen Unvollständigkeit zu rechnen.

Unabhängig davon bestätigt das Ergebnis die bereits intuitiv vermuteten Kategorien Essen und Service. Die Kategorie Musik erscheint etwas zu speziell, dürfte aber in einem breiteren Kontext wie Ambiente/Events/Rahmenprogramm durchaus sinnvoll sein.